# Simple problem -- single city - single order




In [1]:
import pandas as pd
import pulp as plp

In [2]:
input_file = "../tests/data/test00_temlate.xlsx"
output_file = "../data/test_output.xlsx"

## Data

In [3]:
sheet_name = "Parameters"
df_param = pd.read_excel(input_file, sheet_name=sheet_name)
df_param.set_index("ParamName", inplace=True)
minPeriod = df_param.loc["minPeriod"].ParamValue
maxPeriod = df_param.loc["maxPeriod"].ParamValue
minPeriod, maxPeriod

(0, 25)

In [4]:
sheet_name = "Orders"
df_orders = pd.read_excel(input_file, sheet_name=sheet_name)
df_orders.head()

,City,Demand,Price_per_unit,Deliver_from,Deliver_to
0,Tambov,200,800,3,6
1,Voronezh,100,1100,18,22
2,Belgorod,70,1000,10,14
3,Lipetsk,250,750,12,15
4,Ryazan,120,850,9,12


In [5]:
sheet_name = "Routes"
df_routes = pd.read_excel(input_file, sheet_name=sheet_name)
df_routes.head()

,From,To,Time,Distance
0,Voronezh,Lipetsk,1.222222,110
1,Lipetsk,Voronezh,1.222222,110
2,Tambov,Lipetsk,1.333333,120
3,Lipetsk,Tambov,1.333333,120
4,Moscow,Ryazan,2.000000,180


In [6]:
sheet_name = "Cities"
df_cities = pd.read_excel(input_file, sheet_name=sheet_name)
df_cities.head()

,City
0,Moscow
1,Tambov
2,Voronezh
3,Belgorod
4,Lipetsk


In [7]:
sheet_name = "Vehicles"
df_cars= pd.read_excel(input_file, sheet_name=sheet_name, nrows=None)
df_cars.head()

,CarName,Capacity,Price_per_km
0,Nissan,60,9
1,Toyota,50,8
2,Mitsubishi,100,15
3,Mazda,80,10
4,Acura,40,11


In [8]:
TOT_CAPACITY = df_cars.Capacity.sum()
TOT_CAPACITY

330

## Structures

Moscow - origin depot and destination depot

In [9]:
def convert_dataframe_to_dict(dataframe, key_columns, value_column):
    return (
        dataframe.loc[:, key_columns + [value_column]]
        .set_index(key_columns)
        .to_dict()[value_column]
    )

In [10]:
def convert_dataframe_to_set(dataframe, key_columns, value_column):
    return set(
        list(convert_dataframe_to_dict(dataframe, key_columns, value_column).keys())
    )

In [11]:
depot = df_cities.City[0]

cities = set(df_cities.City.values).difference(set([depot]))

#origin_depot = default_depot + '_start'
#destination_depot = default_depot + '_end'

all_cities = cities.union([depot])
depot, all_cities

('Moscow',
 {'Belgorod', 'Lipetsk', 'Moscow', 'Penza', 'Ryazan', 'Tambov', 'Voronezh'})

In [12]:
capacity = convert_dataframe_to_dict(df_cars, ["CarName"], "Capacity")
capacity

{'Nissan': 60, 'Toyota': 50, 'Mitsubishi': 100, 'Mazda': 80, 'Acura': 40}

In [13]:
car_price = convert_dataframe_to_dict(df_cars, ["CarName"], "Price_per_km")
car_price

{'Nissan': 9, 'Toyota': 8, 'Mitsubishi': 15, 'Mazda': 10, 'Acura': 11}

In [14]:
routes = convert_dataframe_to_set(df_routes, ["From", "To"], "Distance")

In [15]:
route_time = convert_dataframe_to_dict(df_routes, ["From", "To"], "Time")
route_distance = convert_dataframe_to_dict(df_routes, ["From", "To"], "Distance")

In [16]:
route_time, route_distance, routes

({('Voronezh', 'Lipetsk'): 1.2222222222222223,
  ('Lipetsk', 'Voronezh'): 1.2222222222222223,
  ('Tambov', 'Lipetsk'): 1.3333333333333333,
  ('Lipetsk', 'Tambov'): 1.3333333333333333,
  ('Moscow', 'Ryazan'): 2.0,
  ('Ryazan', 'Moscow'): 2.0,
  ('Tambov', 'Voronezh'): 2.111111111111111,
  ('Voronezh', 'Tambov'): 2.111111111111111,
  ('Voronezh', 'Belgorod'): 2.4444444444444446,
  ('Belgorod', 'Voronezh'): 2.4444444444444446,
  ('Lipetsk', 'Ryazan'): 2.5,
  ('Ryazan', 'Lipetsk'): 2.5,
  ('Tambov', 'Ryazan'): 2.5555555555555554,
  ('Ryazan', 'Tambov'): 2.5555555555555554,
  ('Penza', 'Tambov'): 2.7777777777777777,
  ('Tambov', 'Penza'): 2.7777777777777777,
  ('Belgorod', 'Lipetsk'): 3.4444444444444446,
  ('Lipetsk', 'Belgorod'): 3.4444444444444446,
  ('Voronezh', 'Ryazan'): 3.7777777777777777,
  ('Ryazan', 'Voronezh'): 3.7777777777777777,
  ('Moscow', 'Lipetsk'): 4.0,
  ('Lipetsk', 'Moscow'): 4.0,
  ('Penza', 'Lipetsk'): 4.166666666666667,
  ('Lipetsk', 'Penza'): 4.166666666666667,
  ('Pe

In [17]:
orders = df_orders.set_index("City")
orders

,Demand,Price_per_unit,Deliver_from,Deliver_to
City,,,,
Tambov,200,800,3,6
Voronezh,100,1100,18,22
Belgorod,70,1000,10,14
Lipetsk,250,750,12,15
Ryazan,120,850,9,12
Penza,300,2000,6,7


In [18]:
TOT_DEMAND = df_orders.Demand.sum()
TOT_DEMAND

1040

In [19]:
cars = convert_dataframe_to_set(df_cars, ["CarName"], "Price_per_km")
cars

{'Acura', 'Mazda', 'Mitsubishi', 'Nissan', 'Toyota'}

In [20]:
# trips - number of trips performed by one vehicle
#trips = list(range(1, len(cities)))
trips = list(range(1, TOT_DEMAND//TOT_CAPACITY + 2))
fin_trip = trips[-1]
trips, fin_trip

([1, 2, 3, 4], 4)

## Task

In [21]:
pbm = plp.LpProblem("VRPTW", plp.LpMaximize)

In [22]:
x = plp.LpVariable.dicts("x", [(k, n, r) for k in cars for n in trips for r in routes], cat=plp.LpBinary)

In [23]:
y = plp.LpVariable.dicts("y", [(k, n, i) for k in cars for n in trips for i in all_cities], cat=plp.LpBinary)

In [24]:
d = plp.LpVariable.dicts("d", [(k, n, i) for k in cars for n in trips for i in cities], cat=plp.LpContinuous, lowBound=0)

In [25]:
#lack
lack = plp.LpVariable.dicts("lack", [i for i in cities], cat=plp.LpContinuous, lowBound=0)

In [26]:
t = plp.LpVariable.dicts("t", [(k, n, i) for k in cars for n in trips for i in all_cities], cat=plp.LpContinuous, lowBound=0)

In [27]:
# end time of the trip n
te = plp.LpVariable.dicts("te", [(k, n) for k in cars for n in trips], cat=plp.LpContinuous, lowBound=0)

In [28]:
# maximize revenue
#revenue = sales_profit - devlivery_cost + expensive_undelivered
pbm += plp.lpSum(orders.loc[i].Price_per_unit * d[k, n, i] for k in cars for n in trips for i in cities) \
    - plp.lpSum(route_distance[r] * car_price[k] * x[k, n, r] for k in cars for n in trips for r in routes) \
    - plp.lpSum(te[k, n] for k in cars for n in trips) \
    - plp.lpSum(100 * orders.loc[i].Price_per_unit * lack[i] for i in cities)

In [29]:
# time constraints
for k in cars:
    pbm += t[k, 1, depot] == 0
    for n in trips:
        if n != fin_trip:
            pbm += te[k, n] <= t[k, n+1, depot]
            pbm += t[k, n, depot] <= t[k, n+1, depot]
        pbm += te[k, n] <= maxPeriod
        
        for i, j in routes:
            if j == depot:
                continue
            pbm += t[k, n, i] + route_time[i, j] <= t[k, n, j] + maxPeriod * (1 - x[k, n, (i, j)])
        for i in cities:
            pbm += t[k, n, i] + route_time[i, depot] <= te[k, n] + maxPeriod * (1 - x[k, n, (i, depot)])
            pbm += t[k, n, i] >= orders.loc[i].Deliver_from * y[k, n, i]
            pbm += t[k, n, i] <= orders.loc[i].Deliver_to * y[k, n, i]

In [30]:
for i in cities:
    # every city visited at least once by one vehicle
    # pbm += plp.lpSum(y[k, n, i] for k in cars for n in trips) >= 1

    # demand
    pbm += plp.lpSum(d[k, n, i] for k in cars for n in trips) == orders.loc[i].Demand - lack[i]
    
    
for k in cars:
    for i in all_cities:
        for n in trips:
            pbm += plp.lpSum(x[k, n, (i, j)] for j in all_cities if i != j) == y[k, n, i]
            pbm += plp.lpSum(x[k, n, (j, i)] for j in all_cities if i != j) == y[k, n, i]
    
    for n in trips:
        pbm += plp.lpSum(d[k, n, i] for i in cities) <= capacity[k] 
        for i in cities:
            pbm += d[k, n, i] <= capacity[k] * y[k, n, i] 

In [31]:
pbm.solve()

PulpSolverError: Pulp: Error while trying to execute C:\Users\voev\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solverdir\cbc\win\64\cbc.exe

In [ ]:
plp.LpStatus[pbm.status], plp.value(pbm.objective)

In [ ]:
revenue = sum(orders.loc[i].Price_per_unit * d[k, n, i].value() for k in cars for n in trips for i in cities) \
    - sum(route_distance[r] * car_price[k] * x[k, n, r].value() for k in cars for n in trips for r in routes)
revenue

In [ ]:
#pbm

## Save result

In [ ]:
cols = ["CarName",
        "TripNumber",
        "City_From", 
        "City_To", 
        "Delivered",
        "Lack",
        "StartTime",
        "ArrivalTime", "Deliver_From", "Deliver_To"]
result = pd.DataFrame(columns=cols)
row_number = 0
for k in cars:
    for n in trips:
        for i, j in routes:
            #print(k, n, i, j, x[k, n, (i, j)].value())
            if x[k, n, (i, j)].value() == 0:
                continue
            result.loc[row_number, cols] = k, n, i, j, \
                                        d[k, n, j].value() if j != depot else None, \
                                        None, \
                                        t[k, n, i].value(), \
                                        t[k, n, j].value() if j != depot else te[k, n].value(), \
                                        orders.loc[j].Deliver_from if j in cities else None, \
                                        orders.loc[j].Deliver_to if j in cities else None
            row_number += 1

            
result = result.sort_values(by=["CarName", "TripNumber", "ArrivalTime"])
#result

In [ ]:
for city in cities:
    curr_lack = orders.loc[city].Demand
    for line in result[result.City_To == city].itertuples():
        curr_lack -= line.Delivered
        result.loc[line.Index, "Lack"] = curr_lack
result

In [ ]:
cols = ["City",
        "Demand",
        "Lack"]
delivery = pd.DataFrame(columns=cols)
row_number = 0
for i in cities:
    delivery.loc[row_number, cols] = i, orders.loc[i].Demand, lack[i].value()
    row_number += 1
delivery